In [100]:
# encoding: utf-8

import csv
import json
import sys
import io
import re

stdout = sys.stdout
reload(sys)
sys.setdefaultencoding("ISO-8859-1")
sys.stdout = stdout

In [107]:
csv_reader = csv.reader(open('data/dbpedia-discussion-archive.csv', 'r'))
file_writer = open('data/dbpedia-discussion-archive-questions.txt', 'w')

def contains_question_word(word):
    question_words = ['what', 'when', 'why', 'which', 'who', 'how', 'whose', 'whom']
    return any(s in word.lower() for s in question_words)

def is_ignored_subject(subject):
    # Removing call for, CFP and Announcements
    if re.search(r'call for|cfp|ann\:', subject, re.IGNORECASE) or subject == '':
        return True
    return False

def is_question_topic(subject, message):
    global question_topics, questions

    if is_ignored_subject(subject):
        return False
    elif subject in question_topics or contains_question_word(subject) or contains_question_word(message):
        if subject not in questions.keys():
            questions[subject] = {
                "thread": []
            }
        questions[subject]['thread'].append(u'' + message)
        return True    
    else:
        return False

question_topics = set()
questions = {}

count = 0
for line in csv_reader:
    if is_question_topic(line[3], line[4]):
        question_topics.add(line[3])
        count += 1

print count
print len(questions.keys())

In [108]:
# Writing all subjects separately to a file
for subject in questions.keys():
    # Remove threads with single message and blank subject not doing this for now
#     if len(questions[subject]['thread']) == 1:
#         print subject
#         del questions[subject]
#         continue
    file_writer.write("%s\n" % subject)
    if subject.startswith('DBpedia query result is'):
        print subject

with open('data/dbpedia-discussion-archive-questions.json', 'w') as fp:
    json.dump(questions, fp)

In [106]:
if re.search(r'cfp|ann:', 'ANN: DBpedia 2016-04-beta release', re.IGNORECASE):
    print "EXISTS"
else:
    print "DOES NOT EXIST"